In [1]:
from selenium import webdriver

In [ ]:
# If you want to use a proxy
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
req_proxy = RequestProxy() #you may get different number of proxy when  you run this at each time
proxies = req_proxy.get_proxy_list() 
PROXY = proxies[0].get_address() # there are different proxies to choose from
webdriver.DesiredCapabilities.CHROME['proxy']={
    "httpProxy":PROXY,
    "ftpProxy":PROXY,
    "sslProxy":PROXY,
    
    "proxyType":"MANUAL",
    
}

In [5]:
driver=webdriver.Chrome(executable_path="C:/Users/nango/selenium/chromedriver")

In [6]:
from lxml import html

In [7]:
import time
def find_page(link):
    listings=[]
    elements=link.xpath('//div[@class="iva-item-content-m2FiN"]/div[@class="iva-item-body-NPl6W"]')
    for element in elements:
        prop_name=element.xpath('./div[@class="iva-item-titleStep-2bjuh"]/a/@title')[0]
        price=element.xpath('./div[@class="iva-item-priceStep-2qRpg"]/span/span/meta[@itemprop="price"]/@content')[0]
        region=element.xpath('./div[@class="iva-item-developmentNameStep-1hr7p"]/div/span/span/text()')
        if len(region)>0:
            region=region[0]
        else:
            region=None
        listings.append({"prop_name":prop_name,"price":price,"region":region})
    return listings

In [8]:
all_listings=[]
for i in range(1,100+1):
    driver.get(f"https://www.avito.ru/rossiya/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p={i}")
    time.sleep(10)
    html_page=driver.page_source
    tree=html.fromstring(html_page)
    listings=find_page(tree)
    if len(listings)>0:
        all_listings.extend(listings)
        time.sleep(10)
    else:
        print(i)
        break

In [1]:
import pandas as pd

In [9]:
df_apartments=pd.DataFrame(all_listings)
df_apartments.to_csv('all_apartments.csv',index=False,encoding='utf-8-sig')

In [11]:
driver.close()

In [2]:
df=pd.read_csv("all_apartments.csv")

In [10]:
import re
df["area_sqm"]=df.prop_name.apply(lambda x: float(re.findall(r",\s(\d.*)\sм²",x)[0].replace(",",".")))

In [11]:
df["price"]=df.price.apply(lambda x: int(x))

In [13]:
new_df=df[['area_sqm','price','region']]

In [14]:
new_df.to_csv("all_apartments_final.csv",index=False,encoding='utf-8-sig')